In [1]:
import numpy as np
import pandas as pd
import os
import math
import re
import subprocess
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from pathlib import Path


2021-12-16 15:22:32.655401: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-16 15:22:32.655452: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Helpers


In [2]:
def plotHistory(history):
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(loss))
    plt.plot(epochs, loss, label="Training loss")
    plt.plot(epochs, val_loss, label="Validation loss")
    plt.legend()
    plt.title('Training and validation loss')


In [3]:
checkpoint_path = './checkpoints/lab3/'


def get_cp_callback(path):
    return tf.keras.callbacks.ModelCheckpoint(filepath=path,
                                              save_weights_only=True,
                                              save_freq="epoch",
                                              verbose=1)


def purge_checkpoints():
    if os.path.isdir(checkpoint_path):
        subprocess.run(['rm', '-rf', checkpoint_path])
        subprocess.run(['mkdir', checkpoint_path])


def purge_text(text):
    text = text.lower()
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub("<.*?>+|[^a-zA-Z\s]", "", text)
    text = " ".join(filter(lambda x: x[0] != "@", text.split()))
    return text


# Data processing


In [4]:
df_true = pd.read_csv("./data/lab3/True.csv")

df = df_true.drop(["subject", "date"], axis=1).drop(
    [i for i in range(int(len(df_true) / 300), len(df_true))], axis=0)


df['text'] = df['text'].apply(lambda x: purge_text(x))
df['title'] = df['title'].apply(lambda x: purge_text(x))
df


,title,text
0,as us budget fight looms republicans flip thei...,washington reuters the head of a conservative ...
1,us military to accept transgender recruits on ...,washington reuters transgender people will be ...
2,senior us republican senator let mr mueller do...,washington reuters the special counsel investi...
3,fbi russia probe helped by australian diplomat...,washington reuters trump campaign adviser geor...
4,trump wants postal service to charge much more...,seattlewashington reuters president donald tru...
...,...,...
66,in georgia battle of the staceys tests democra...,atlanta reuters the two democratic candidates ...
67,democrat wins by one vote in virginia legislat...,corrects spelling of virginia house of delegat...
68,fbi deputy director to sit for closed intervie...,washington reuters the fbis deputy director an...
69,white house expects congress to waive spending...,washington reuters the white house expects the...


In [5]:
text_data = []

for i in df.index:
    text_data.append(str(df.iloc[i, 0]) + ":" + str(df.iloc[i, 1]) + "\n")


In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1
input_sequences = tokenizer.texts_to_sequences(text_data)
print("Total words: " + str(total_words))


Total words: 4752


In [7]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)
labels = tf.keras.utils.to_categorical(input_sequences, num_classes=total_words)

train_len = math.ceil(input_sequences.shape[0] * 0.7)
val_len = math.ceil(input_sequences.shape[0] * 0.3)

train_ds = (input_sequences[:train_len], labels[:train_len])
val_ds = (input_sequences[val_len:], labels[val_len:])


# Model definition

In [8]:
checkpoint_path = "./checkpoints/lab3/init/"
purge_checkpoints()

model = models.Sequential([
    layers.Embedding(total_words, 128, input_length=max_sequence_length),
    layers.GRU(128, return_sequences=True),
    layers.Dropout(0.2),
    layers.Dense(256, activation='relu'),
    layers.Dense(total_words, activation='softmax')
])

2021-12-16 15:22:37.649234: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-16 15:22:37.649280: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-16 15:22:37.649310: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Arch): /proc/driver/nvidia/version does not exist
2021-12-16 15:22:37.650728: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(optimizer="adam",
              loss='categorical_crossentropy')

history = model.fit(train_ds,
                    validation_data=val_ds,
                    callbacks=[get_cp_callback(checkpoint_path)],
                    epochs=10,
                    use_multiprocessing=True,
                    workers=4)

plotHistory(history)
model.save('./models/lab3/init/')

2021-12-16 15:22:37.977091: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1220313600 exceeds 10% of free system memory.


Epoch 1/10


ValueError: in user code:

    File "/home/pacifi5t/Projects/University/neural-networks/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/pacifi5t/Projects/University/neural-networks/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pacifi5t/Projects/University/neural-networks/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/pacifi5t/Projects/University/neural-networks/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/pacifi5t/Projects/University/neural-networks/.venv/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_